# Metadata training

Idea: use MLP on metadata to predict labels.

First, process labels into one-hot encoded vector.

In [18]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read data

In [23]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir = os.path.join(parent_directory, 'data')
intermediate_data_dir = os.path.join(data_dir, 'intermediate_output')

In [25]:
labels_df = pd.read_pickle(os.path.join(intermediate_data_dir, 'labels.pkl'))
labels_df.head()

,mbtag,multi_hot
song_id,,
SOVVDCO12AB0187AF7,"[fusion, jazz fusion, classic pop and rock]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
SOKSZVC12A8C142004,[finnish],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
SOZQSGL12AF72A9145,"[punk, california, san francisco, rock and ind...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
SOBHQIJ12A6D4FC7D3,"[old school hip-hop, hip-hop, american, rock a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
SODENZR12AAF3B13B9,[folk],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [26]:
music_analysis_df = pd.read_pickle(os.path.join(intermediate_data_dir, 'music_analysis.pkl'))
music_analysis_df.head()

,analysis/bars_start,analysis/beats_start,analysis/segments_start,analysis/sections_start,analysis/segments_loudness_max,analysis/segments_pitches,analysis/segments_timbre,analysis/tatums_start
song_id,,,,,,,,
SOVVDCO12AB0187AF7,"[0.86028, 3.32671, 5.78675, 8.25191, 10.70588,...","[0.26495, 0.86028, 1.47917, 2.10185, 2.71162, ...","[0.0, 0.43252, 0.60086, 0.94916, 1.34971, 2.02...","[0.0, 20.4191, 60.63045, 97.02151, 152.90716, ...","[-60.0, -33.028, -25.443, -42.479, -15.58, -17...","[[0.253, 0.295, 0.256, 0.255, 0.365, 0.256, 0....","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,...","[0.11365, 0.26495, 0.41625, 0.56755, 0.71126, ..."
SOKSZVC12A8C142004,"[1.1801, 2.5338, 3.8774, 5.21764, 6.50406, 7.8...","[0.50326, 0.84168, 1.1801, 1.51853, 1.86368, 2...","[0.0, 0.44893, 0.56481, 0.81737, 0.90463, 1.40...","[0.0, 18.94707, 37.60065, 50.59071, 82.07264, ...","[-59.127, -55.639, -30.079, -20.537, -5.212, -...","[[0.818, 1.0, 0.803, 0.887, 0.758, 0.796, 0.76...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,...","[0.50326, 0.84168, 1.1801, 1.51853, 1.86368, 2..."
SOZQSGL12AF72A9145,"[1.04773, 3.659, 6.31974, 8.97532, 11.61875, 1...","[0.39241, 1.04773, 1.69575, 2.34571, 3.00003, ...","[0.0, 0.19837, 0.49914, 0.85873, 1.01193, 1.17...","[0.0, 6.98708, 31.7588, 55.79356, 71.87843, 10...","[-23.305, -22.35, -22.875, -28.333, -20.214, -...","[[0.146, 0.203, 0.219, 0.209, 0.172, 0.123, 0....","[[23.928, -119.362, 33.96, -122.038, 8.715, 18...","[0.06784, 0.23214, 0.39241, 0.5559, 0.7194, 0...."
SOBHQIJ12A6D4FC7D3,"[1.53651, 3.73492, 5.93365, 8.15439, 10.3851, ...","[0.21007, 0.65246, 1.09485, 1.53651, 1.98112, ...","[0.0, 0.17891, 0.33379, 0.61823, 0.90853, 1.05...","[0.0, 13.03524, 69.72052, 99.80305, 112.67158,...","[-60.0, -15.641, -7.938, -7.652, -9.136, -10.3...","[[1.0, 0.284, 0.224, 0.217, 0.245, 0.285, 0.38...","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,...","[0.0626, 0.21007, 0.35753, 0.505, 0.65246, 0.7..."
SODENZR12AAF3B13B9,"[0.09527, 0.70899, 1.3226, 1.93617, 2.55139, 3...","[0.09527, 0.70899, 1.3226, 1.93617, 2.55139, 3...","[0.0, 0.34676, 0.61247, 0.70635, 0.87283, 1.22...","[0.0, 8.5233, 32.22853, 49.1158, 61.85519, 75....","[-60.0, -26.53, -27.841, -23.783, -24.037, -32...","[[0.203, 0.125, 0.086, 0.095, 0.088, 0.266, 0....","[[0.0, 171.13, 9.469, -28.48, 57.491, -50.067,...","[0.09527, 0.4029, 0.70899, 1.0228, 1.3226, 1.6..."


In [27]:
metadata_df = pd.read_pickle(os.path.join(intermediate_data_dir, 'metadata.pkl'))
metadata_df.head()

,duration,end_of_fade_in,key,loudness,mode,start_of_fade_out,tempo,time_signature
song_id,,,,,,,,
SOVVDCO12AB0187AF7,279.35302,0.433,9,-12.474,1,261.288,99.024,4
SOKSZVC12A8C142004,255.03302,0.449,9,-4.393,1,251.275,175.673,4
SOZQSGL12AF72A9145,216.84200,0.000,10,-4.264,1,213.020,92.897,4
SOBHQIJ12A6D4FC7D3,393.63873,0.179,1,-6.619,1,379.478,135.503,5
SODENZR12AAF3B13B9,146.33751,0.347,1,-25.651,0,137.143,104.989,1
